In [ ]:
# import required packages
from __future__ import print_function, division
#import cv2
import dlib
import time
from skimage import io
import os
import shutil
import torch
import pandas as pd
import torchvision
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.stats.api as sms
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from os import listdir
from os.path import isfile, join
from collections import namedtuple
from skimage.transform import rescale
from skimage.transform import resize
import sys
import glob
import PIL
import os
import shutil
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn.functional as F
import math
import PIL
import imgaug as ia
from imgaug import augmenters as iaa
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from pathlib import Path
from collections import Counter
import imgaug as ia
from imgaug import augmenters as iaa
import cv2
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode
print(os.getcwd())
#np.random.seed(198467)
torch.cuda.empty_cache()

In [2]:
frame = pd.read_csv('ExpW_OpenFace_result_with_expression_gender_race_age.csv')
frame = frame[['ImageName', 'Expression', 'gender_preds']]
frame.gender_preds = frame.gender_preds.apply(lambda x: 1 if x == "Male" else 0)
frame = frame[['ImageName', 'Expression', 'gender_preds']]
frame.Expression = frame.Expression.apply(lambda x: 1 if x == 3 else 0)
frame.head()

,ImageName,Expression,gender_preds
0,angry_actor_104_0.jpg,0,1
1,angry_actor_109_0.jpg,0,1
2,angry_actor_120_1.jpg,0,1
3,angry_actor_13_0.jpg,1,0
4,angry_actor_132_0.jpg,0,0


In [3]:
frame.shape

(88600, 3)

In [4]:
frame.Expression.value_counts()

0    59707
1    28893
Name: Expression, dtype: int64

In [5]:
frame.gender_preds.value_counts()

1    61108
0    27492
Name: gender_preds, dtype: int64

In [6]:
frame['ImageName'] = frame['ImageName'].apply(lambda x: '/data/ExpW_cropped_by_their_coordinates/' + x) # change ImageName to full path
frame_copy = frame.copy()

In [7]:
result_rows_list = []

In [8]:
for k in range(1,6):

    # Split into train/validation/test sets
    frame = frame_copy.sample(n = 20000, random_state = k).reset_index(drop=True) # shuffle data frame
    n_images = len(frame)
    n_train = int(0.8 * n_images)
    n_val = int((n_images - n_train) / 2)
    n_test = n_images - n_train - n_val

    train_frame = frame[0 : n_train].reset_index(drop=True)
    val_frame = frame[n_train : n_train + n_val].reset_index(drop=True)
    test_frame = frame[n_train + n_val : ].reset_index(drop=True)

    print("{} train faces, {} validation faces, {} test faces".format(len(train_frame), len(val_frame), len(test_frame)))

    # Data loaders and transforms for training

    # Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
    # e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second
    # image.
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)

    # Define our sequence of augmentation steps that will be applied to every image.
    class ImgAugTransform:

        def __init__(self):
            self.aug = iaa.Sequential(
            [
                #
                # Apply the following augmenters to most images.
                #
                iaa.Fliplr(0.5), # horizontally flip 50% of all images
                #iaa.Flipud(0.2), # vertically flip 20% of all images

                # crop some of the images by 0-10% of their height/width
                sometimes(iaa.Crop(percent=(0, 0.05))),

                # Apply affine transformations to some of the images
                # - scale to 80-120% of image height/width (each axis independently)
                # - translate by -20 to +20 relative to height/width (per axis)
                # - rotate by -45 to +45 degrees
                # - shear by -16 to +16 degrees
                # - order: use nearest neighbour or bilinear interpolation (fast)
                # - mode: use any available mode to fill newly created pixels
                #         see API or scikit-image for which modes are available
                # - cval: if the mode is constant, then use a random brightness
                #         for the newly created pixels (e.g. sometimes black,
                #         sometimes white)
                iaa.Affine(
                    scale={"x": (1, 1.1), "y": (1, 1.1)}, # scale images to 80-120% of their size, individually per axis
                    translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, # translate by -10 to +10 percent (per axis)
                    rotate=(-15, 15), # rotate by -15 to +15 degrees
                    shear=(-8, 8), # shear by -8 to +8 degrees
                    order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
                    #cval=(0, 255), # if mode is constant, use a cval between 0 and 255
                    mode=['edge'] # use any of scikit-image's warping modes (see 2nd image from the top for examples)
                ),

                #
                # Execute 0 to 5 of the following (less important) augmenters per
                # image. Don't execute all of them, as that would often be way too
                # strong.
                #
                iaa.SomeOf((0, 5),
                    [
                        # Convert some images into their superpixel representation,
                        # sample between 20 and 200 superpixels per image, but do
                        # not replace all superpixels with their average, only
                        # some of them (p_replace).
                        sometimes(
                            iaa.Superpixels(
                                p_replace=(0, 0.1),
                                n_segments=(50, 200)
                            )
                        ),

                        # Blur each image with varying strength using
                        # gaussian blur (sigma between 0 and 3.0),
                        # average/uniform blur (kernel size between 2x2 and 7x7)
                        # median blur (kernel size between 3x3 and 11x11).
                        iaa.OneOf([
                            iaa.GaussianBlur((0, 3.0)),
                            iaa.AverageBlur(k=(2, 7)),
                            iaa.MedianBlur(k=(3, 11)),
                        ]),

                        # Sharpen each image, overlay the result with the original
                        # image using an alpha between 0 (no sharpening) and 1
                        # (full sharpening effect).
                        iaa.Sharpen(alpha=(0, 0.3), lightness=(0.75, 1.5)),

                        # Same as sharpen, but for an embossing effect.
                        iaa.Emboss(alpha=(0, 0.3), strength=(0, 2)),

                        # Search in some images either for all edges or for
                        # directed edges. These edges are then marked in a black
                        # and white image and overlayed with the original image
                        # using an alpha of 0 to 0.7.
                        sometimes(iaa.OneOf([
                            iaa.EdgeDetect(alpha=(0, 0.3)),
                            iaa.DirectedEdgeDetect(
                                alpha=(0, 0.3), direction=(0.0, 1.0)
                            ),
                        ])),

                        # Add gaussian noise to some images.
                        # In 50% of these cases, the noise is randomly sampled per
                        # channel and pixel.
                        # In the other 50% of all cases it is sampled once per
                        # pixel (i.e. brightness change).
                        iaa.AdditiveGaussianNoise(
                            loc=0, scale=(0.0, 0.05*255), per_channel=0.5
                        ),

                        # Either drop randomly 1 to 10% of all pixels (i.e. set
                        # them to black) or drop them on an image with 2-5% percent
                        # of the original size, leading to large dropped
                        # rectangles.
                        iaa.OneOf([
                            iaa.Dropout((0.01, 0.02), per_channel=0.5),
                            #iaa.CoarseDropout(
                            #    (0.03, 0.15), size_percent=(0.02, 0.05),
                            #    per_channel=0.2
                            #),
                        ]),

                        # Invert each image's chanell with 5% probability.
                        # This sets each pixel value v to 255-v.
                        #iaa.Invert(0.05, per_channel=True), # invert color channels

                        # Add a value of -10 to 10 to each pixel.
                        iaa.Add((-15, 15), per_channel=0.5),

                        # Change brightness of images (50-150% of original value).
                        iaa.Multiply((0.75, 1.25), per_channel=0.5),

                        # Improve or worsen the contrast of images.
                        iaa.ContrastNormalization((0.75, 1.75), per_channel=0.5),

                        # Convert each image to grayscale and then overlay the
                        # result with the original with random alpha. I.e. remove
                        # colors with varying strengths.
                        iaa.Grayscale(alpha=(0.0, 1.0)),

                        # In some images move pixels locally around (with random
                        # strengths).
                        #sometimes(
                        #    iaa.ElasticTransformation(alpha=(0.1, 0.2), sigma=0.25)
                        #),

                        # In some images distort local areas with varying strength.
                        sometimes(iaa.PiecewiseAffine(scale=(0.005, 0.01)))
                    ],
                    # do all of the above augmentations in random order
                    random_order=True
                )
            ],
            # do all of the above augmentations in random order
            random_order=True
        )

        def __call__(self, img):

            img = np.array(img)
            return self.aug.augment_image(img)

    class ImageDataset(Dataset):

        def __init__(self, data_frame, transform=None):

            self.data_frame = data_frame
            self.transform = transform

        def __len__(self):

            return len(self.data_frame)

        def __getitem__(self, idx):
            #idx is index from dataset
            #This is a mapping from your data_frame to the output of the mode
            img_name = self.data_frame.loc[idx, 'ImageName']

            expression = self.data_frame.loc[idx, 'Expression']
            gender = self.data_frame.loc[idx, 'gender_preds']

            # read image as ndarray, H*W*C
            image = dlib.load_rgb_image(img_name)       
            image = cv2.resize(image, (224,224)) # resize the image to 224x224 for the ResNet Model

            if self.transform:
                image = self.transform(image)

            # transform label to torch tensor
            # This sets the order of the label
            return (image, torch.from_numpy(np.asarray(expression, dtype=np.float32)), 
                    torch.from_numpy(np.asarray(gender, dtype=np.float32)))


    transform_train_data = transforms.Compose([
        ImgAugTransform(),
        lambda x: PIL.Image.fromarray(x),
        transforms.ToTensor(), 
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    transformed_train_dataset = ImageDataset(data_frame=train_frame,
                                               transform=transform_train_data
                                               )

    train_dataloader = DataLoader(transformed_train_dataset, batch_size=32,
                            shuffle=False, num_workers=8)


    transform_val_data = transforms.Compose(([transforms.ToPILImage(), 
                                          transforms.ToTensor(), 
                                          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                              ]))

    transformed_val_dataset = ImageDataset(data_frame=val_frame,
                                               transform=transform_val_data
                                               )

    val_dataloader = DataLoader(transformed_val_dataset, batch_size=32,
                            shuffle=False, num_workers=8)

    transform_test_data = transforms.Compose(([transforms.ToPILImage(), 
                                          transforms.ToTensor(), 
                                          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                              ]))

    transformed_test_dataset = ImageDataset(data_frame=test_frame,
                                               transform=transform_test_data
                                               )

    test_dataloader = DataLoader(transformed_test_dataset, batch_size=32,
                            shuffle=False, num_workers=8)

    # Training

    torch.cuda.is_available()

    dataloaders = {'train': train_dataloader, 'test': test_dataloader}
    dataset_sizes = {'train': len(transformed_train_dataset), 'test': len(transformed_test_dataset)}
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # utils.py

    import json
    import pickle
    from sklearn.metrics import average_precision_score

    def save_pkl(pkl_data, save_path):
        with open(save_path, 'wb') as f:
            pickle.dump(pkl_data, f)

    def load_pkl(load_path):
        with open(load_path, 'rb') as f:
            pkl_data = pickle.load(f)
        return pkl_data

    def save_json(json_data, save_path):
        with open(save_path, 'w') as f:
            json.dump(json_data, f)

    def load_json(load_path):
        with open(load_path, 'r') as f:
            json_data = json.load(f)
        return json_data

    def save_state_dict(state_dict, save_path):
        torch.save(state_dict, save_path)

    def creat_folder(path):
        if not os.path.exists(path):
            os.makedirs(path)

    def set_random_seed(seed_number):
        torch.manual_seed(seed_number)
        np.random.seed(seed_number)

    def write_info(filename, info):
        with open(filename, 'w') as f:
            f.write(info)

    def compute_weighted_AP(target, predict_prob, class_weight_list):
        per_class_AP = []
        for i in range(target.shape[1] - 1):
            class_weight = target[:, i]*class_weight_list[i] \
                           + (1-target[:, i])*np.ones(class_weight_list[i].shape)
            per_class_AP.append(average_precision_score(target[:, i], predict_prob[:], 
                                    sample_weight=class_weight))

        return per_class_AP

    def compute_mAP(per_class_AP):
        return np.mean([per_class_AP[idx] for idx in [0]])

    def compute_class_weight(target):
        domain_label = target[:, -1]
        per_class_weight = []

        for i in range(target.shape[1]-1):
            class_label = target[:, i]
            cp = class_label.sum() # class is positive
            cn = target.shape[0] - cp # class is negative
            cn_dn = ((class_label + domain_label)==0).sum() # class is negative, domain is negative
            cn_dp = ((class_label - domain_label)==-1).sum()
            cp_dn = ((class_label - domain_label)==1).sum()
            cp_dp = ((class_label + domain_label)==2).sum()

            per_class_weight.append(
                (class_label*cp + (1-class_label)*cn) / 
                    (2*(
                        (1-class_label)*(1-domain_label)*cn_dn
                        + (1-class_label)*domain_label*cn_dp
                        + class_label*(1-domain_label)*cp_dn
                        + class_label*domain_label*cp_dp
                       )
                    )
            )
        return per_class_weight

    # celeba_gradproj_adv.py
    class ResNet50(nn.Module):    
        def __init__(self, n_classes, pretrained, hidden_size=2048, dropout=0.5):
            super().__init__()
            self.resnet = torchvision.models.resnet50(pretrained=pretrained)                
            self.resnet.fc = nn.Linear(2048, hidden_size)
            self.fc = nn.Linear(hidden_size, n_classes)
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(dropout)        

        def require_all_grads(self):
            for param in self.parameters():
                param.requires_grad = True

        def forward(self, x):
            features = self.resnet(x)
            outputs = self.fc(self.dropout(self.relu(features)))

            return outputs, features


    class CelebaGradProjAdv():
        def __init__(self):
            self.training_ratio = 1
            self.alpha = 0.1
            self.epoch = 0
            self.best_dev_mAP = 0.
            self.train_loader = train_dataloader    
            self.dev_loader = val_dataloader        
            self.test_loader = test_dataloader

            self.dev_target = val_frame.iloc[:,[1,2]].to_numpy()
            self.dev_class_weight = compute_class_weight(self.dev_target)
            self.test_target = test_frame.iloc[:,[1,2]].to_numpy()
            self.test_class_weight = compute_class_weight(self.test_target)


            self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

            self.class_network = ResNet50(n_classes=1, pretrained=True).to(self.device)


            self.domain_network = nn.Linear(1, 2).to(self.device)

            num_female = len(train_frame[train_frame.gender_preds == 0])
            num_male = len(train_frame[train_frame.gender_preds == 1])
            weight_gender = torch.tensor([num_male/num_female, num_male/num_male]).to(self.device)

            self.domain_criterion = nn.CrossEntropyLoss(weight = weight_gender)

            num_happy = len(train_frame[train_frame.Expression == 1])
            num_nothappy = len(train_frame[train_frame.Expression == 0])
            weight_happy = torch.tensor([num_nothappy/num_happy]).to(self.device)

            self.criterion = nn.BCEWithLogitsLoss(pos_weight = weight_happy)

            self.class_optimizer = torch.optim.Adam( 
                                      params=filter(lambda p: p.requires_grad, self.class_network.parameters()), 
                                      lr=1e-4)
            self.domain_optimizer = torch.optim.Adam( 
                                      params=filter(lambda p: p.requires_grad, self.domain_network.parameters()), 
                                      lr=1e-4)


        def _criterion(self, output, expression):
            return self.criterion(torch.squeeze(output), expression.float())
            #return F.binary_cross_entropy_with_logits(torch.squeeze(output), expression.float())


        def _domain_criterion(self, output, gender):

            return self.domain_criterion(torch.squeeze(output), gender.long())


        def state_dict(self):
            state_dict = {
                'class_network': self.class_network.state_dict(),
                'domain_network': self.domain_network.state_dict(),
                'class_optimizer': self.class_optimizer.state_dict(),
                'domain_optimizer': self.domain_optimizer.state_dict(),
                'epoch': self.epoch
            }
            return state_dict

        def load_state_dict(self, state_dict):
            self.class_network.load_state_dict(state_dict['class_network'])
            self.domain_network.load_state_dict(state_dict['domain_network'])


        def inference(self, output):
            predict_prob = torch.sigmoid(output)
            return predict_prob.cpu().numpy()


        def _train(self, loader):
            """Train the model for one epoch"""

            self.class_network.train()
            self.domain_network.train()

            train_class_loss = 0
            train_domain_loss = 0
            total = 0
            class_correct = 0
            domain_correct = 0
            for i, (images, expression, gender) in enumerate(loader):
                images, expression, gender = images.to(self.device), expression.to(self.device), gender.to(self.device)


                self.class_optimizer.zero_grad()
                self.domain_optimizer.zero_grad()


                class_outputs, _ = self.class_network(images)
                domain_outputs = self.domain_network(class_outputs)
                class_outputs = torch.squeeze(class_outputs)

                class_loss = self._criterion(class_outputs, expression)
                domain_loss = self._domain_criterion(domain_outputs, gender)


                total += expression.size(0)

                class_predicted = torch.where(torch.sigmoid(class_outputs) >= 0.5, torch.ones_like(class_outputs), torch.zeros_like(class_outputs)).long()
                class_correct += class_predicted.eq(expression.long()).sum().item()
                _, domain_predicted = domain_outputs.max(1)
                domain_correct += domain_predicted.eq(gender.long()).sum().item()


                if i == 0 and self.epoch % 4 == 0:
                    print()
                    print("class outputs:")
                    print(class_outputs)
                    print("class predicted")
                    print(class_predicted)
                    print("class loss")
                    print(class_loss)
                    print()
                    print("domain outputs:")
                    print(domain_outputs)
                    print("domain predicted")
                    print(domain_predicted)
                    print("domain loss")
                    print(domain_loss)



                # Update the domain classifier
                #domain_grad = torch.autograd.grad(domain_loss, self.domain_network.parameters(), 
                #                                  retain_graph=True, allow_unused=True)
                #for param, grad in zip(self.domain_network.parameters(), domain_grad):
                #    param.grad = grad.clone()
                #domain_loss.backward(retain_graph=True)
                #self.domain_optimizer.step()


                # Update the main network
                if self.epoch % self.training_ratio == 0:
                    #grad_from_class = torch.autograd.grad(class_loss, self.class_network.parameters(),
                    #                                      retain_graph=True, allow_unused=True)
                    #grad_from_domain = torch.autograd.grad(domain_loss, self.class_network.parameters(),
                    #                                       retain_graph=True, allow_unused=True)
                    #torch.autograd.set_detect_anomaly(True)
                    grad_from_class = torch.autograd.grad(class_loss, self.class_network.parameters(), retain_graph=True)
                    grad_from_domain = torch.autograd.grad(domain_loss, self.class_network.parameters(), retain_graph=True)
                    
                    
                    # Update the domain classifier
                    domain_grad = torch.autograd.grad(domain_loss, self.domain_network.parameters(), 
                                                  retain_graph=True, allow_unused=True)
                    for param, grad in zip(self.domain_network.parameters(), domain_grad):
                        param.grad = grad.clone()
                    self.domain_optimizer.step()
                    
                    
                    # Update the main network
                    for param, class_grad, domain_grad in zip(self.class_network.parameters(), 
                                                              grad_from_class, grad_from_domain):
                        if (class_grad is not None) and (domain_grad is not None): 
                            # Gradient projection
                            if domain_grad.norm() > 1e-5:
                                param.grad = class_grad.clone() - self.alpha*domain_grad.clone() - \
                                        ((class_grad.clone()*domain_grad.clone()).sum()/domain_grad.clone().norm()) \
                                        * (domain_grad.clone()/domain_grad.clone().norm()) 
                            else:
                                param.grad = class_grad.clone() - self.alpha*domain_grad.clone() 

                            #if domain_grad.norm() > 1e-5:
                            #    param.grad = class_grad - self.alpha*domain_grad - \
                            #            ((class_grad*domain_grad).sum()/domain_grad.norm()) \
                            #            * (domain_grad/domain_grad.norm()) 
                            #else:
                            #    param.grad = class_grad - self.alpha*domain_grad 
                    self.class_optimizer.step()

                train_class_loss += class_loss.item()
                train_domain_loss += domain_loss.item()
                #self.log_result('Train iteration', 
                #                {'class_loss': class_loss.item(),
                #                 'domain_loss': domain_loss.item(),
                #                 'domain_accuracy': 100.*correct/total},
                #                len(loader)*self.epoch + i)

                #if self.print_freq and (i % self.print_freq == 0):
            print('Training epoch {}: [{}|{}], class loss:{}, class accuracy: {}, domain loss: {}, domain accuracy: {}'
                          .format(self.epoch, i+1, len(loader), 
                                  class_loss.item(), 100.*class_correct/total, domain_loss.item(),
                                  100.*domain_correct/total))

            #self.log_result('Train epoch', 
            #                {'class_loss': train_class_loss/len(loader),
            #                 'domain_loss': train_domain_loss/len(loader),
            #                 'domain_accuracy': 100.*correct/total}, 
            #                self.epoch)
            self.epoch += 1



        def _test(self, loader):
            """Compute model output on test set"""

            self.class_network.eval()
            self.domain_network.eval()

            test_class_loss = 0
            test_domain_loss = 0
            total = 0
            class_correct = 0
            domain_correct = 0
            feature_list = []
            class_output_list = []
            domain_output_list = []
            with torch.no_grad():
                for i, (images, expression, gender) in enumerate(loader):
                    images, expression, gender = images.to(self.device), expression.to(self.device), gender.to(self.device)

                    class_outputs, features = self.class_network(images)
                    domain_outputs = self.domain_network(class_outputs)
                    class_outputs = torch.squeeze(class_outputs)

                    class_loss = self._criterion(class_outputs, expression)
                    domain_loss = self._domain_criterion(domain_outputs, gender)
                    test_class_loss += class_loss.item()
                    test_domain_loss += domain_loss.item()

                    total += expression.size(0)

                    class_predicted = torch.where(torch.sigmoid(class_outputs) >= 0.5, torch.ones_like(class_outputs), torch.zeros_like(class_outputs)).long()
                    class_correct += class_predicted.eq(expression.long()).sum().item()

                    _, domain_predicted = domain_outputs.max(1)
                    domain_correct += domain_predicted.eq(gender.long()).sum().item()

                    class_output_list.append(class_outputs)
                    domain_output_list.append(domain_outputs)
                    feature_list.append(features)

                return test_class_loss, test_domain_loss, torch.cat(class_output_list), 100.*class_correct/total, \
                       torch.cat(domain_output_list), torch.cat(feature_list), 100.*domain_correct/total



        def public_test(self, images):
            """Compute model output on test set"""

            self.class_network.eval()
            self.domain_network.eval()

            class_outputs, _ = self.class_network(images)
            domain_outputs = self.domain_network(class_outputs)
            class_outputs = torch.squeeze(class_outputs)

            return torch.sigmoid(class_outputs), domain_outputs


        def train(self):
            """Train the model for one epoch, evaluate on validation set and 
            save the best model
            """

            start_time = datetime.now()
            learning_rates1 = [1e-5, 1e-6, 1e-6] # class_optimizer
            learning_rates2 = [1e-3, 5e-4, 1e-4] # domain_optimizer
            epochs = [8, 4, 2]
            
            for learning_rate1, learning_rate2, epoch in zip(learning_rates1, learning_rates2, epochs):
                print()
                print('learning rate:', learning_rate1)
                print('epoch number:', epoch)
                self.class_optimizer = torch.optim.Adam( 
                                          params=filter(lambda p: p.requires_grad, self.class_network.parameters()), 
                                          lr=learning_rate1, weight_decay=1e-5)
                self.domain_optimizer = torch.optim.Adam( 
                                          params=filter(lambda p: p.requires_grad, self.domain_network.parameters()), 
                                          lr=learning_rate2, weight_decay=1e-5)
                for i in range(epoch):
                    print()
                    self._train(self.train_loader)

                    dev_class_loss, dev_domain_loss, dev_class_output, dev_class_accuarcy, _,_, dev_domain_accuarcy = self._test(self.dev_loader)
                    dev_predict_prob = self.inference(dev_class_output)
                    dev_per_class_AP = compute_weighted_AP(self.dev_target, dev_predict_prob, 
                                                                 self.dev_class_weight)
                    dev_mAP = compute_mAP(dev_per_class_AP)

                    print('Testing epoch, class loss:{},  class accuracy: {}, mAP: {}, domain loss: {}, domain accuracy: {}'
                              .format(dev_class_loss/len(self.dev_loader), dev_class_accuarcy, dev_mAP,  
                                      dev_domain_loss/len(self.dev_loader), dev_domain_accuarcy))



                    #self.log_result('Dev epoch', 
                    #                {'class_loss': dev_class_loss/len(self.dev_loader), 
                    #                 'domain_loss': dev_domain_loss/len(self.dev_loader),
                    #                 'mAP': dev_mAP,
                    #                 'domain_accuracy': dev_domain_accuarcy},
                    #                self.epoch)
                    #if dev_mAP > self.best_dev_mAP:
                    #    self.best_dev_mAP = dev_mAP
                    #
                    #    best_model_wts = copy.deepcopy(self.state_dict())
                    #    
                    #    
                    #    save_state_dict(self.state_dict(), os.path.join("./test/", 'best.pth'))
                    #if
                    #    print('best mAP, loss update:', dev_class_accuarcy, self.best_dev_mAP)
                    #else:
                    #    print('best mAP, loss not update, still:', dev_class_accuarcy, self.best_dev_mAP)

                #self.load_state_dict(best_model_wts)
                save_state_dict(self.state_dict(), os.path.join("./", 'gradproj_' + str(k) + '.pth'))
            duration = datetime.now() - start_time
            print('Finish training epoch {}, dev class loss: {}, dev doamin loss: {}, dev mAP: {},'\
                  'domain_accuracy: {}, time used: {}'
                  .format(self.epoch, dev_class_loss/len(self.dev_loader), 
                          dev_domain_loss/len(self.dev_loader), dev_mAP, dev_domain_accuarcy,
                          duration))



        def test(self):
            # Test and save the result
            state_dict = torch.load(os.path.join("./", 'gradproj_' + str(k) + '.pth'))
            self.load_state_dict(state_dict)



            test_class_loss, test_domain_loss, test_class_output, test_class_accuarcy, test_domain_output, \
                test_feature, test_domain_accuracy = self._test(self.test_loader)
            test_predict_prob = self.inference(test_class_output)
            test_per_class_AP = compute_weighted_AP(self.test_target, test_predict_prob, 
                                                         self.test_class_weight)
            test_mAP = compute_mAP(test_per_class_AP)
            print('per_class_AP', test_per_class_AP)
            print('mAP', test_mAP)
            print('domain_accuracy', test_domain_accuracy)
            return test_class_output.cpu().numpy(), test_domain_output.cpu().numpy(), test_feature.cpu().numpy()



    model = CelebaGradProjAdv()
    model.train()



    # Evaluate on Test Set

    # predict on test

    #model = CelebaGradProjAdv()
    #model.load_state_dict(torch.load('./test/best_gradproj.pth'))

    trans = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    face_names = []
    scores = []
    preds = []

    true_labels = []

    gender_preds = []
    true_gender_labels = []

    for index, row in test_frame.iterrows():

        if index % 200 == 0:
            print(index)

        image_name = row['ImageName']

        image = dlib.load_rgb_image(image_name)
        image = trans(image)
        image = image.view(1, 3, 224, 224)
        image = image.to(device)

        outputs, gender_outputs = model.public_test(image)
        outputs = outputs.cpu().detach().numpy()
        outputs = np.squeeze(outputs)

        score = outputs * 1
        #score = np.exp(outputs) / np.sum(np.exp(outputs))
        pred = (score>=0.5)*1
        #pred = np.argmax(score)

        gender_outputs = gender_outputs.cpu().detach().numpy()
        gender_outputs = np.squeeze(gender_outputs)

        gender_score = np.exp(gender_outputs) / np.sum(np.exp(gender_outputs))
        gender_pred = np.argmax(gender_score)

        face_names.append(image_name)
        scores.append(score)
        preds.append(pred)
        gender_preds.append(gender_pred)

        true_labels.append(row['Expression'])
        true_gender_labels.append(row['gender_preds'])

    test_result = pd.DataFrame(list(zip(face_names, scores, preds, true_labels, gender_preds, true_gender_labels)), 
                 columns = ['ImageName', 'ExpressionScore', 'Prediction', 'Expression', 'GenderPrediction', 'Gender'])
    test_result.head(10)

    # Accuracy
    test_result['CorrectOrNot'] = (test_result.Prediction == test_result.Expression)
    test_result['CorrectOrNot_gender'] = (test_result.GenderPrediction == test_result.Gender)


    dict_row = {}
    dict_row['model_random_state'] = k
    dict_row['test_accuracy'] = test_result.CorrectOrNot.mean()
    dict_row['test_gender_accuracy'] = test_result.CorrectOrNot_gender.mean()
    dict_row['test_male_true_proportion'] = (test_result[test_result.Gender == 1].Expression==1).mean()
    dict_row['test_female_true_proportion'] = (test_result[test_result.Gender == 0].Expression==1).mean()
    dict_row['test_male_predicted_proportion'] = (test_result[test_result.Gender == 1].Prediction==1).mean()
    dict_row['test_female_predicted_proportion'] = (test_result[test_result.Gender == 0].Prediction==1).mean()
    dict_row['test_male_average_score'] = test_result[test_result.Gender == 1].ExpressionScore.mean()
    dict_row['test_female_average_score'] = test_result[test_result.Gender == 0].ExpressionScore.mean()



    # Apply on Chicago Face

    chicago_df = pd.read_csv('ChicagoFace_selected_evaluation_set2.csv').iloc[:,0:4]
    chicago_df.head()

    def expression_string_to_num(s):
        if s == "Happy":
            return 3
        elif s == "Angry":
            return 0
        elif s == "Fear":
            return 2
        else:
            return 6

    chicago_df["Expression_num"] = chicago_df.Expression.apply(expression_string_to_num)


    chicago_df["happy"] = chicago_df.Expression_num.apply(lambda x: 1 if x == 3 else 0)




    trans = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    face_names = []
    scores = []
    preds = []

    true_labels = []

    gender_preds = []

    for index, row in chicago_df.iterrows():

        #if index >= 100: break

        if index % 100 == 0:
            print(index)

        image_name = row['ImageName']


        image = dlib.load_rgb_image(image_name)
        image = trans(image)
        image = image.view(1, 3, 224, 224)
        image = image.to(device)

        outputs, gender_outputs = model.public_test(image)
        outputs = outputs.cpu().detach().numpy()
        outputs = np.squeeze(outputs)

        score = outputs * 1
        #score = np.exp(outputs) / np.sum(np.exp(outputs))
        pred = (score>=0.5)*1
        #pred = np.argmax(score)

        gender_outputs = gender_outputs.cpu().detach().numpy()
        gender_outputs = np.squeeze(gender_outputs)

        gender_score = np.exp(gender_outputs) / np.sum(np.exp(gender_outputs))
        gender_pred = np.argmax(gender_score)

        face_names.append(image_name)
        scores.append(score)
        preds.append(pred)
        gender_preds.append(gender_pred)

        true_labels.append(row['happy'])

    chicago_result = pd.DataFrame([face_names, preds, scores, gender_preds]).T

    chicago_result.columns = ['ImageName', 'expression_preds', 'expression_scores', 'gender_preds_model']


    chicago_result["happiness_score"] = chicago_result.expression_scores.apply(lambda x: x)

    num_happy = (chicago_df.happy == 1).sum()
    threshold = chicago_result.sort_values(by = ['happiness_score'], ascending = False).reset_index(drop = True).iloc[num_happy]['happiness_score']

    chicago_result['expression_preds_relabeled'] = chicago_result.happiness_score.apply(lambda x : 1 if x > threshold else 0)


    chicago_df_merged = pd.merge(chicago_df, chicago_result, on = ['ImageName'], how = 'left')
    chicago_df_merged["true_gender"] = chicago_df_merged.Gender.apply(lambda x: 1 if x == "M" else 0)


    chicago_df_merged['CorrectOrNot'] = (chicago_df_merged.expression_preds_relabeled == chicago_df_merged.happy)
    dict_row['selected_2_chicago_accuracy'] = chicago_df_merged.CorrectOrNot.mean() 

    dict_row['selected_2_chicago_gender_accuracy'] = (chicago_df_merged.gender_preds_model == chicago_df_merged.true_gender).mean()

    chicago_male_frame = chicago_df_merged.loc[chicago_df_merged['Gender'] == 'M']
    chicago_female_frame = chicago_df_merged.loc[chicago_df_merged['Gender'] == 'F']

    # Accuracy between males and females
    dict_row['selected_2_chicago_accuracy_male'] = (chicago_male_frame.expression_preds_relabeled == chicago_male_frame.happy).mean()
    dict_row['selected_2_chicago_accuracy_female'] = (chicago_female_frame.expression_preds_relabeled == chicago_female_frame.happy).mean()

    # True proportion
    dict_row['selected_2_chicago_true_proportion'] = chicago_male_frame.happy.mean()

    # Prediction proportion
    dict_row['selected_2_chicago_male_predicted_proportion'] = chicago_male_frame.expression_preds_relabeled.mean()
    dict_row['selected_2_chicago_female_predicted_proportion'] = chicago_female_frame.expression_preds_relabeled.mean()
    dict_row['selected_2_chicago_bias'] = chicago_female_frame.expression_preds_relabeled.mean() - chicago_male_frame.expression_preds_relabeled.mean()

    # Prediction proportion
    dict_row['selected_2_chicago_male_predicted_proportion_raw'] = chicago_male_frame.expression_preds.mean()
    dict_row['selected_2_chicago_female_predicted_proportion_raw'] = chicago_female_frame.expression_preds.mean()
    dict_row['selected_2_chicago_bias_raw'] = chicago_female_frame.expression_preds.mean() - chicago_male_frame.expression_preds.mean()

    # Average Happiness Score
    dict_row['selected_2_chicago_male_score'] = chicago_male_frame.happiness_score.mean()
    dict_row['selected_2_chicago_female_score'] = chicago_female_frame.happiness_score.mean()

    # Average Happiness Score among neutral faces
    dict_row['selected_2_chicago_male_score_neutral_faces'] = chicago_male_frame[chicago_male_frame.Expression_num == 6].happiness_score.mean()
    dict_row['selected_2_chicago_female_score_neutral_faces'] = chicago_female_frame[chicago_female_frame.Expression_num == 6].happiness_score.mean()


    result_rows_list.append(dict_row)
    results_df = pd.DataFrame(result_rows_list) 
    print(results_df)
    results_df.to_csv('model_evaluation_result.csv', index = False)

    del model
    torch.cuda.empty_cache()



16000 train faces, 2000 validation faces, 2000 test faces

learning rate: 1e-05
epoch number: 8


class outputs:
tensor([ 0.1025, -0.0807, -0.0125,  0.0199, -0.1976, -0.1700, -0.0619,  0.0773,
         0.2234, -0.0678, -0.2115,  0.0689, -0.1896, -0.0800, -0.0397,  0.2279,
        -0.0333, -0.0736, -0.0124,  0.0445, -0.2495,  0.0499, -0.1010, -0.2012,
         0.0857,  0.0047,  0.0921, -0.0384, -0.1332,  0.2484, -0.0322, -0.1694],
       device='cuda:0', grad_fn=<SqueezeBackward0>)
class predicted
tensor([1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
        1, 1, 1, 0, 0, 1, 0, 0], device='cuda:0')
class loss
tensor(1.0077, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

domain outputs:
tensor([[-0.3235,  0.5684],
        [-0.1570,  0.6431],
        [-0.2190,  0.6153],
        [-0.2485,  0.6020],
        [-0.0509,  0.6908],
        [-0.0760,  0.6795],
        [-0.1742,  0.6354],
        [-0.3006,  0.5786],
        [-0.4334,  0.5190],
        

Training epoch 9: [500|500], class loss:0.000181619543582201, class accuracy: 99.9, domain loss: 0.7040476202964783, domain accuracy: 58.4625
Testing epoch, class loss:2.254676057232751,  class accuracy: 82.2, mAP: 0.8127615771456643, domain loss: 0.692135078566415, domain accuracy: 62.2

Training epoch 10: [500|500], class loss:0.00010862501221708953, class accuracy: 99.94375, domain loss: 0.703599214553833, domain accuracy: 59.1
Testing epoch, class loss:2.2970253881954013,  class accuracy: 82.15, mAP: 0.8126921018502163, domain loss: 0.6921738338848901, domain accuracy: 62.4

Training epoch 11: [500|500], class loss:0.00011734259169315919, class accuracy: 99.9875, domain loss: 0.7043686509132385, domain accuracy: 59.48125
Testing epoch, class loss:2.34144431376268,  class accuracy: 82.25, mAP: 0.8128584806212686, domain loss: 0.6921564350052486, domain accuracy: 62.35

learning rate: 1e-06
epoch number: 2


class outputs:
tensor([-25.8949,   8.5960, -17.0502,  -8.1506, -16.8641,   8


class outputs:
tensor([  6.8878,  -5.6349,   3.9852,   7.5372,  -2.9950,  -5.5266,  -2.8470,
         -5.3956,  -4.4754, -13.2152,  -4.6623, -14.4397,  -3.6059,  -6.1785,
         -4.9341,   9.4447,  -6.1708,  13.2326,  -4.5186,  -2.2586,   7.0474,
         10.9237,  -6.9443,  -3.4658, -14.1488,  -6.5172,   5.9177,  10.3417,
         -4.1921,  -7.6657,  -4.2897,  -6.8228], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
class predicted
tensor([1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 0], device='cuda:0')
class loss
tensor(0.0125, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

domain outputs:
tensor([[  5.9159,   5.7849],
        [ -4.1254,  -4.0613],
        [  3.5884,   3.5026],
        [  6.4366,   6.2955],
        [ -2.0086,  -1.9856],
        [ -4.0385,  -3.9761],
        [ -1.8899,  -1.8693],
        [ -3.9335,  -3.8732],
        [ -3.1956,  -3.1496],
        [-10.2037, -10.0214],
        [ -3.3455

Testing epoch, class loss:2.10745140626317,  class accuracy: 84.0, mAP: 0.8433940629476901, domain loss: 0.692495082105909, domain accuracy: 64.8

Training epoch 13: [500|500], class loss:7.341351010836661e-05, class accuracy: 100.0, domain loss: 0.6915121078491211, domain accuracy: 61.61875
Testing epoch, class loss:2.1929179361415287,  class accuracy: 84.15, mAP: 0.8422764029921207, domain loss: 0.6929742362764146, domain accuracy: 69.5
Finish training epoch 14, dev class loss: 2.1929179361415287, dev doamin loss: 0.6929742362764146, dev mAP: 0.8422764029921207,domain_accuracy: 69.5, time used: 0:50:34.680870
0
200
400
600
800
1000
1200
1400
1600
1800
0
100
200
300
400
500
600
700
800
900
1000
1100
0
100
200
300
400
500
0
100
200
300
400
500
600
700
   model_random_state  test_accuracy  test_gender_accuracy  \
0                   1         0.8200                0.6280   
1                   2         0.8345                0.6705   

   test_male_true_proportion  test_female_true_prop

Testing epoch, class loss:1.3378913707676388,  class accuracy: 84.7, mAP: 0.8379915442775547, domain loss: 0.689018794468471, domain accuracy: 59.6

Training epoch 6: [500|500], class loss:0.0639462098479271, class accuracy: 99.48125, domain loss: 0.6951084136962891, domain accuracy: 57.20625
Testing epoch, class loss:1.5060524748904365,  class accuracy: 84.75, mAP: 0.8248695761054549, domain loss: 0.6924579569271633, domain accuracy: 44.5

Training epoch 7: [500|500], class loss:0.0268157497048378, class accuracy: 99.5375, domain loss: 0.6958786249160767, domain accuracy: 54.625
Testing epoch, class loss:1.4847372077287189,  class accuracy: 86.0, mAP: 0.8526549443988858, domain loss: 0.693545727502732, domain accuracy: 32.0

learning rate: 1e-06
epoch number: 4


class outputs:
tensor([ -3.3368,  13.5703, -58.3719,   5.0519,  -3.0159,  -7.6959,  -3.4105,
         13.6602,  11.4120,  -4.9472,   7.2398,  -0.5307, -15.6020,  -6.4137,
         -7.0681,   6.7776,   0.2124,  -8.8898,   9.91


learning rate: 1e-05
epoch number: 8


class outputs:
tensor([ 0.1572, -0.0429,  0.3460,  0.1576, -0.0458,  0.3208, -0.1948,  0.0770,
         0.1840, -0.0814, -0.1035, -0.2290, -0.0505,  0.0684, -0.1725, -0.0118,
        -0.2457, -0.0131, -0.1173, -0.0656,  0.0654,  0.1321, -0.1822, -0.0254,
         0.0427,  0.0414, -0.0039, -0.0507, -0.1176, -0.2584,  0.0465,  0.0707],
       device='cuda:0', grad_fn=<SqueezeBackward0>)
class predicted
tensor([1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        1, 1, 0, 0, 0, 0, 1, 1], device='cuda:0')
class loss
tensor(0.9631, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

domain outputs:
tensor([[ 0.2652, -0.4411],
        [ 0.2095, -0.5244],
        [ 0.3178, -0.3625],
        [ 0.2653, -0.4409],
        [ 0.2086, -0.5256],
        [ 0.3108, -0.3730],
        [ 0.1671, -0.5877],
        [ 0.2429, -0.4745],
        [ 0.2727, -0.4299],
        [ 0.1987, -0.5405],
        [ 0.1926, -0.5496],
        [ 

Training epoch 9: [500|500], class loss:0.0004649396287277341, class accuracy: 99.94375, domain loss: 0.6920441389083862, domain accuracy: 60.31875
Testing epoch, class loss:2.075833720820291,  class accuracy: 83.4, mAP: 0.7984746108217509, domain loss: 0.6947765066510155, domain accuracy: 41.6

Training epoch 10: [500|500], class loss:0.000483524810988456, class accuracy: 99.9875, domain loss: 0.6925938129425049, domain accuracy: 61.66875
Testing epoch, class loss:2.117943947277372,  class accuracy: 83.4, mAP: 0.7988250862950711, domain loss: 0.694800215108054, domain accuracy: 41.55

Training epoch 11: [500|500], class loss:0.00033527836785651743, class accuracy: 99.9875, domain loss: 0.6932030916213989, domain accuracy: 60.95
Testing epoch, class loss:2.144238018327289,  class accuracy: 83.4, mAP: 0.7988353690795228, domain loss: 0.6947568721241422, domain accuracy: 41.75

learning rate: 1e-06
epoch number: 2


class outputs:
tensor([-10.3907,  13.4702,  -6.1764, -17.1571,  -9.6982,

Training epoch 0: [500|500], class loss:0.5462560653686523, class accuracy: 68.6875, domain loss: 0.6982300281524658, domain accuracy: 51.4125
Testing epoch, class loss:0.5513738819531032,  class accuracy: 82.4, mAP: 0.8376546915558988, domain loss: 0.6831514314999656, domain accuracy: 56.85

Training epoch 1: [500|500], class loss:0.29426485300064087, class accuracy: 84.35625, domain loss: 0.6969358921051025, domain accuracy: 57.94375
Testing epoch, class loss:0.5614879547603546,  class accuracy: 83.0, mAP: 0.8404601720281556, domain loss: 0.6865135260990688, domain accuracy: 57.6

Training epoch 2: [500|500], class loss:0.04347431659698486, class accuracy: 94.10625, domain loss: 0.6972671151161194, domain accuracy: 58.8625
Testing epoch, class loss:0.77741301509123,  class accuracy: 82.85, mAP: 0.824470546777343, domain loss: 0.6886334589549473, domain accuracy: 58.4

Training epoch 3: [500|500], class loss:0.013036094605922699, class accuracy: 99.10625, domain loss: 0.69658309221267

Training epoch 12: [500|500], class loss:0.00039488705806434155, class accuracy: 100.0, domain loss: 0.6951507329940796, domain accuracy: 64.2375
Testing epoch, class loss:1.843626786389255,  class accuracy: 85.05, mAP: 0.8338976589396089, domain loss: 0.6926631870723906, domain accuracy: 64.95

Training epoch 13: [500|500], class loss:0.0001466775720473379, class accuracy: 100.0, domain loss: 0.6955587863922119, domain accuracy: 65.03125
Testing epoch, class loss:1.9610161220170705,  class accuracy: 85.0, mAP: 0.833486336746176, domain loss: 0.692798754525563, domain accuracy: 65.3
Finish training epoch 14, dev class loss: 1.9610161220170705, dev doamin loss: 0.692798754525563, dev mAP: 0.833486336746176,domain_accuracy: 65.3, time used: 0:50:28.920774
0
200
400
600
800
1000
1200
1400
1600
1800
0
100
200
300
400
500
600
700
800
900
1000
1100
0
100
200
300
400
500
0
100
200
300
400
500
600
700
   model_random_state  test_accuracy  test_gender_accuracy  \
0                   1         0